In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Cargamos las funciones necesarias

In [ ]:
import numpy as np 
import pandas as pd
import calendar
import matplotlib.pyplot as plt
import seaborn as sns
import squarify
import gc
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import time
import datetime
from datetime import datetime
import calendar

from sklearn import model_selection # model assesment and model selection strategies
from sklearn import metrics # model evaluation metrics
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.tree import export_graphviz
import graphviz
from sklearn.preprocessing import LabelEncoder,MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb


sns.set_style('white')

pd.options.display.float_format = '{:,.2f}'.format

Definimos la funcion que nos hara las diferencias mensuales entre los productos EasyMoney

In [ ]:
def calcula_diferencias_mensuales (dataset, variable):
    dataset[variable+'_pm']  = dataset.groupby('pk_cid')[variable].shift(1)
    dataset['dif_'+variable] = dataset[variable] - dataset[variable+'_pm']
    #dataset['dif_'+variable]  = dataset.groupby('pk_cid')[variable].diff()
    dataset.drop(variable+'_pm',axis=1,inplace=True)


Cargamos los tres dataset Necesarios

In [ ]:
products_file = '/kaggle/input/easymoney/products_df.csv'
products = pd.read_csv(products_file)
products.drop('Unnamed: 0', axis=1, inplace=True)

sd_file = '/kaggle/input/easymoney/sociodemographic_df.csv'
sociodemographic = pd.read_csv(sd_file)
sociodemographic.drop('Unnamed: 0',axis=1, inplace=True)

ca_file = '/kaggle/input/easymoney/commercial_activity_df.csv'
commercial = pd.read_csv(ca_file)
commercial.drop('Unnamed: 0',axis=1, inplace=True)

Los unimos de forma adecuada:

In [ ]:
df_= pd.merge(products,commercial, how="inner",on=['pk_cid','pk_partition' ])
df=pd.merge(df_,sociodemographic, how="inner",on=['pk_cid','pk_partition'])

In [ ]:
#df_sorted = pd.read_pickle('/kaggle/input/easymoney/EasyMoney_base.pkl',compression='zip')

# Obtenemos las diferencias por meses:

Ordenamos el Dataset y llamamos a la función calcula_diferencias_mensuales

In [ ]:
df_sorted = df.sort_values(by=['pk_cid', 'pk_partition'])

In [ ]:
del products, sociodemographic, commercial, df

In [ ]:
gc.collect()

In [ ]:
productos_easymoney=['loans',
 'mortgage',
 'funds',
 'securities',
 'long_term_deposit',
 'em_account_pp',
 'credit_card',
 'payroll',
 'pension_plan',
 'payroll_account',
 'emc_account',
 'debit_card',
 'em_account_p',
 'em_acount']

In [ ]:
for x in productos_easymoney:
    calcula_diferencias_mensuales (df_sorted, x)

# Creamos campo es usuario nuevo:

In [ ]:
# las ponemos a 2019-02-28
df_sorted.loc[ (df_sorted['entry_date']=='2019-02-29'), 
              'entry_date']='2019-02-28'
df_sorted.loc[ (df_sorted['entry_date']=='2015-02-29'), 
              'entry_date']='2015-02-28'

In [ ]:
# Vamos a ponerlos como fechas
for i in ["pk_partition","entry_date"]:
    df_sorted[i]=pd.to_datetime(df_sorted[i], format='%Y-%m-%d')

In [ ]:
# Vamos a restar las dos y lo ponemos en mesess:
df_sorted['mesesAlta']=(df_sorted['pk_partition']-df_sorted['entry_date'])/np.timedelta64(1,'M')


In [ ]:
# Creamos el campo isNewClient
df_sorted['isNewClient']=((df_sorted['mesesAlta'] < 1) & 
                          (df_sorted['mesesAlta'] > 0)).astype(int)

In [ ]:
# Cuando es usuario nuevo todos los campos diff de ses mes estan a 0. Pero el puede haber contratado 
# algo en ese mismo mes y no estaria recogido en el campo diff. 
# Para las altas nuevas igualamos los dif con los contadores del producto
for x in productos_easymoney:
    df_sorted.loc[ (df_sorted['isNewClient']==1) &
                   (df_sorted['dif_'+x].isnull()==True), 
                  'dif_'+x]=df_sorted[x]

# Creamos campo es usuario activo:

In [ ]:
df_sorted['isActive']=((df_sorted['loans']==0) &
                        (df_sorted['mortgage']==0) &
                        (df_sorted['funds']==0) &
                        (df_sorted['securities']==0) &
                        (df_sorted['long_term_deposit']==0) &
                        (df_sorted['em_account_pp']==0) &
                        (df_sorted['credit_card']==0) &
                        (df_sorted['payroll']==0) &
                        (df_sorted['pension_plan']==0) &
                        (df_sorted['payroll_account']==0) &
                        (df_sorted['emc_account']==0) &
                        (df_sorted['debit_card']==0) &
                        (df_sorted['em_account_p']==0) &
                        (df_sorted['em_acount']==0)).astype(int)
# pero nos queda al reves, hacemos la negacion
df_sorted['isActive']=(df_sorted['isActive']!=1).astype(int)

In [ ]:
#df_sorted.to_pickle('./dataset_base.pkl')

In [ ]:
#df_sorted =pd.read_pickle('./dataset_base.pkl')

In [ ]:
df_sorted.info(verbose=False)

# Variable totalAssest

In [ ]:
# Easymoney product list
productos_easymoney=['loans',
 'mortgage',
 'funds',
 'securities',
 'long_term_deposit',
 'em_account_pp',
 'credit_card',
 'payroll',
 'pension_plan',
 'payroll_account',
 'emc_account',
 'debit_card',
 'em_account_p',
 'em_acount']

In [ ]:
df_assets=df_sorted.melt(id_vars=['pk_partition','pk_cid'],
              value_vars=productos_easymoney,
              var_name='Product',
              value_name='Count')

In [ ]:
df_assets=df_assets.groupby(['pk_partition','pk_cid']).agg({'Count':np.sum}).reset_index(drop=False)

In [ ]:
df_assets.rename(columns={'Count':'totalAssets'}, inplace=True)

In [ ]:
len(df_sorted),len(df_assets)

In [ ]:
df_sorted=pd.merge(df_sorted,df_assets, how="inner",on=['pk_cid','pk_partition'])

In [ ]:
del df_assets

# Cargamos directamente el dataset anterior del pickle

In [ ]:
# Leememos directamente todo lo anterior del pickle generado en el notebook: Easymoney_first_steps:
df_sorted = pd.read_pickle('/kaggle/input/easymoney/EasyMoney_base.pkl',compression='zip')

In [ ]:
df_sorted.isnull().sum()

In [ ]:
df_sorted['payroll'].fillna(0,inplace=True)

In [ ]:
df_sorted['pension_plan'].fillna(0,inplace=True)

In [ ]:
df_sorted

Para poder hacer predicciones para el mes 06/2019 del que no tenemos datos, vamos a cambiar de forma al dataset. Vamos a pasar todas las columnas dif_ con los incrementos mensuales a solo dos columnas "producto" y "count" mediante un melt del dataset. Dejamos em_acount y el debit_card para luego poder sacar mejor las variables LAG. Hacemos el melt solo para dos productos por limitaciones de memoria

In [ ]:
vars_colums=['pk_cid','pk_partition',
'entry_date',             
'entry_channel',                 
'active_customer',              
'segment',                     
'country_id',                  
'region_code',                  
'gender',                         
'age',                            
'deceased',              
'salary',  
'mesesAlta',                   
'isNewClient',                    
'isActive',
'em_acount',
'totalAssets']
#'debit_card'] 


delta_productos_easymoney=['dif_loans',
 'dif_mortgage',
 'dif_funds',
 'dif_securities',
 'dif_long_term_deposit',
 'dif_em_account_pp',
 'dif_credit_card',
 'dif_payroll',
 'dif_pension_plan',
 'dif_payroll_account',
 'dif_emc_account',
 'dif_debit_card',
 'dif_em_account_p',
 'dif_em_acount']

In [ ]:
delta_productos_easymoney=[
 'dif_em_acount']

In [ ]:
df_altas=df_sorted.melt(id_vars=vars_colums,
              value_vars=delta_productos_easymoney,
              var_name='Product',
              value_name='Count')

In [ ]:
df_altas.isnull().sum()

In [ ]:
del df_sorted

In [ ]:
gc.collect()

# Vamos a hacer el producto cartesiano para predecir
Solo clientes activos(Con producto) el ultimo mes

Vamos a tratar de hacer la prediccion de altas/bajas solo para los clientes activos en el ultimo mes del que tenemos datos y solo para el producto em_acount por limitaciones de memoria. Añadimos el mes 2019-06-28 a la lista de fechas para tener datos de ese mes

In [ ]:
lista_fechas=['2018-01-28',
'2018-02-28',
'2018-03-28',
'2018-04-28',
'2018-05-28',
'2018-06-28',
'2018-07-28',
'2018-08-28',
'2018-09-28',
'2018-10-28',
'2018-11-28',
'2018-12-28',
'2019-01-28',
'2019-02-28',
'2019-03-28',
'2019-04-28',
'2019-05-28',             
'2019-06-28']

In [ ]:
# Solo clientes activos el ultimo mes:
lista_clientes=df_altas[(df_altas['isActive']==1) & 
                        (df_altas['pk_partition']=='2019-05-28')]["pk_cid"].unique().tolist()

In [ ]:
# Solo uno por razones de memoria del Kernel
lista_productos=['dif_em_acount']

Hacemos el producto cartesiano:

In [ ]:
cartesian_product = pd.MultiIndex.from_product([lista_fechas, 
                                                lista_clientes , 
                                                lista_productos], names = ["pk_partition", "pk_cid", "Product"])
len(cartesian_product)

In [ ]:
cartesian_product

In [ ]:
full_df = pd.DataFrame(index = cartesian_product).reset_index()
full_df.tail()

In [ ]:
# Ponemos el campo pk_partition a tipo fecha
full_df['pk_partition']=pd.to_datetime(full_df['pk_partition'], format='%Y-%m-%d')

In [ ]:
full_df.groupby('pk_partition')['pk_cid'].size()

Vemos que esta OK, con 331588 clientes en cada mes. 

Hace:mos el merge de los datos 

In [ ]:
full_df = pd.merge(full_df, df_altas, on = ["pk_partition", "pk_cid", "Product"], how = 'left')

In [ ]:
del df_altas

In [ ]:
full_df.groupby('pk_partition')['pk_cid'].size()

Debido al producto cartesiano se nos generra muchos clientes inactivos en las fechas posteriores. Asi que procedo a borrar los clientes de "relleno" en los meses que no existian y que no nos aportan informacion. De hecho sin hacer este paso, el desempeño del modelo es muy, muy malo. 

In [ ]:
# Borramos todos los nulos que se generan por el producto cartesiano:
full_df.drop (full_df[ (full_df['pk_partition']!='2019-06-28') &
                       (full_df['entry_date'].isnull()) ].index, axis=0, inplace=True)

In [ ]:
# Comprobamos el borrado:
full_df.groupby('pk_partition')['pk_cid'].size()

In [ ]:
len(full_df)

Borramos los meses en que no pasa nada, para que el modleo aprenda emjor los cambios:

In [ ]:
full_df.drop(full_df[(full_df['isNewClient']==0) &
                        (full_df['isActive']==0) & 
                       (full_df['em_acount']==0) &
                       (full_df['Count']==0) ].index ,axis=0, inplace=True)

Comprobamos que se han borado clientes en los meses antiguos

In [ ]:
full_df.groupby('pk_partition')['pk_cid'].size()

In [ ]:
full_df[(full_df['pk_cid']==1231342)]

In [ ]:
full_df.info(verbose=False)

En el mes a predecir la información relativa al cliente esta toda a NAN, como es logico. La repoblamos en la medida de  lo posible con los datos de meses anteriores:

In [ ]:
lista_actualizar=['entry_date',
 #'entry_channel',
 'active_customer',
 #'segment',
 #'country_id',
 'region_code',
 #'gender',
 'age',
 #'deceased',
 'salary']
 #'mesesAlta']

In [ ]:
# Rellenamos entry_date y los campos anteriores para el ultimo mes 06/2019:
for x in lista_actualizar:
    print(x)
    full_df.loc[(full_df['pk_partition']=='2019-06-28'),
                x]=full_df[full_df['pk_partition']=='2019-06-28']['pk_cid'].map(full_df[['pk_cid',x]].groupby('pk_cid')[x].max())

In [ ]:
full_df[(full_df['pk_cid']==1231342) & (full_df['pk_partition']=='2019-06-28')]

Hay que volver sobre este punto porque podemos poblar mas campos, con datos antiguos. 

In [ ]:
# Solo hay nulos en el ultimo mes ,el mes a predecir:
full_df.isnull().sum()

In [ ]:
#full_df.to_pickle('./full_df_antesFe.pkl')

In [ ]:
#full_df =pd.read_pickle('./full_df_antesFe.pkl')

In [ ]:
gc.collect()

# Feature engeniering

Vamos a sacar variables LAG con los valores de los meses anteriores que son las que deberemos de usar de forma mayoritaria para entrenar al modelo

In [ ]:
# Solo 5 meses hacia atras por razones de memoria del kernel
for x in [1,2,3,4,5]:
    full_df['Count_shift_'+str(x)]=full_df.groupby(['pk_cid','Product'])['Count'].shift(x)
    full_df['em_acount_shift_'+str(x)]=full_df.groupby(['pk_cid','Product'])['em_acount'].shift(x)
    full_df['isActive_shift_'+str(x)]=full_df.groupby(['pk_cid','Product'])['isActive'].shift(x)
    full_df['isNewClient_shift_'+str(x)]=full_df.groupby(['pk_cid','Product'])['isNewClient'].shift(x)
    full_df['active_customer_shift_'+str(x)]=full_df.groupby(['pk_cid','Product'])['active_customer'].shift(x)
    full_df['totalAssets_shift_'+str(x)]=full_df.groupby(['pk_cid','Product'])['totalAssets'].shift(x)

In [ ]:
# La borro xq no aporta mucho y hacemos espacio:
full_df.drop('country_id',axis=1,inplace=True)

In [ ]:
full_df.fillna(-999, inplace=True)

Sacos variables de la fecha del mes en curso

In [ ]:
full_df["year"] = full_df["pk_partition"].dt.year
full_df["month"] = full_df["pk_partition"].dt.month

In [ ]:
full_df["entry_date_year"] = full_df["entry_date"].dt.year
full_df["entry_date_month"] = full_df["entry_date"].dt.month

Hot encoding con el genero

In [ ]:
dummy_dataset = pd.get_dummies(full_df['gender'],prefix='gender')

In [ ]:
dummy_dataset.head()

In [ ]:
full_df = pd.concat([full_df,dummy_dataset],axis=1)

In [ ]:
del dummy_dataset

In [ ]:
gc.collect()

In [ ]:
full_df.isnull().sum().sum()

In [ ]:
#full_df.to_pickle('./full_df_antesFe.pkl')

# Entrenar modelo

Vamos a entrenar con todos los meses excepto los dos ultimos. Validamos con el penultimo y test en el ultimo. Todavia nos quedaria el del mes 06/2019 que seria del que ya realmente no tenemos datos. Quito los primeros meses del entrenamiento donde las variables lags son nulas (-999)

In [ ]:
train_index = sorted(list(full_df["pk_partition"].unique()))[6:-3]

valida_index = [sorted(list(full_df["pk_partition"].unique()))[-3]]

test_index = [sorted(list(full_df["pk_partition"].unique()))[-2]]

Borramos las variables que hacen referencia al mes en curso y solo entrenamos el modelo con las LAGS y con las propias del cliente

In [ ]:
variables_borrar= ['Count' ,
"pk_partition",
'pk_cid',
'Product',
'segment',
'gender', 
'deceased',
'mesesAlta',
'entry_date',
#'country_id',
'entry_channel',
'em_acount',
#'debit_card',
'isNewClient',
'isActive',
'active_customer',
'totalAssets']

In [ ]:
X_train = full_df[full_df["pk_partition"].isin(train_index)].drop(variables_borrar, axis=1)
Y_train = full_df[full_df["pk_partition"].isin(train_index)]['Count']

X_valida = full_df[full_df["pk_partition"].isin(valida_index)].drop(variables_borrar, axis=1)
Y_valida = full_df[full_df["pk_partition"].isin(valida_index)]['Count']

X_test = full_df[full_df["pk_partition"].isin(test_index)].drop(variables_borrar, axis=1)
Y_test = full_df[full_df["pk_partition"].isin(test_index)]['Count']

In [ ]:
del full_df

In [ ]:
gc.collect()

In [ ]:
dt = DecisionTreeClassifier(max_depth=7,random_state=42)

In [ ]:
dt.fit(X_train,Y_train)

In [ ]:
score_train=dt.score(X_train, Y_train)
score_test=dt.score(X_valida, Y_valida)
print('Resultados para: Train: {} - Test: {}'.format(score_train,score_test))


Parece un Score muy bueno, pero si lo vemos en detalle es muy pobre

In [ ]:
y_valida_pred = pd.DataFrame(dt.predict(X_valida), index=Y_valida.index, columns=['CountPrediction'])

In [ ]:
results_df = Y_valida.to_frame().join(y_valida_pred)

In [ ]:
results_df[results_df['Count']!=0].sample(40)

Vemos que el modelo parece cazar bien los 0 y los 1, pero no las bajas. 

In [ ]:
results_df['error']=results_df['Count']-results_df['CountPrediction']

In [ ]:
results_df['error'].hist()

In [ ]:
results_df[results_df['Count']==0]['error'].hist()

In [ ]:
results_df[results_df['Count']==-1]['error'].hist()

Caza muy pocas bajas!!!!!!!!!!!!!!!!!!!!! Solo unas pocas mas de 200

In [ ]:
results_df[results_df['Count']==1]['error'].hist()

> ![](http://)Las altas mucho mejor. Cazamos 3475 altas y fallamos 552. Lo que nos da un 84% de acierto en las altas. [](http://)

In [ ]:
results_df[results_df['Count']!=0]['error'].value_counts()

In [ ]:
top_features = pd.Series(dt.feature_importances_, index=X_train.columns).sort_values(ascending=False).head(20)

In [ ]:
top_features

In [ ]:
len(results_df)

In [ ]:
results_df[results_df['Count']==0]['error'].value_counts()

De los casos que son 0 (Se queda como esta), se aciertan 321419 y se fallan 1369

In [ ]:
results_df[results_df['Count']==-1]['error'].value_counts()

Se los casos que son baja, se aciertan 272 y se fallan 1707

In [ ]:
results_df[results_df['Count']==1]['error'].value_counts()

De los que son altas de una nueva cuenta se aciertan 3379 y se fallan 552.